**Entrega 2 – Preprocesamiento de datos (Competencia Pruebas Saber PRO Colombia)**

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from google.colab import files
import io


In [2]:
from google.colab import files

# Subir el archivo desde tu computador
uploaded = files.upload()

import io
import pandas as pd




Saving train.csv to train.csv


In [8]:
# Cargar el archivo que acabas de subir
df = pd.read_csv(io.BytesIO(uploaded['train.csv']), low_memory=False)

print("✅ train.csv cargado correctamente.")
print("Dimensiones:", df.shape)


✅ train.csv cargado correctamente.
Dimensiones: (692500, 21)


Inspección inicial de los datos

In [9]:
# Vista rápida de los datos
df.head()


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [10]:
# Información general
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

In [11]:
# Valores faltantes por columna
df.isnull().sum().sort_values(ascending=False).head(15)


,0
F_TIENEAUTOMOVIL,43623
F_TIENELAVADORA,39773
F_TIENECOMPUTADOR,38103
F_ESTRATOVIVIENDA,32137
E_HORASSEMANATRABAJA,30857
F_TIENEINTERNET.1,26629
F_TIENEINTERNET,26629
F_EDUCACIONMADRE,23664
F_EDUCACIONPADRE,23178
E_PAGOMATRICULAPROPIO,6498


Limpieza de datos

In [12]:
# Eliminar columnas con muchos valores nulos (si aplica)
threshold = 0.5  # Porcentaje máximo permitido de nulos
cols_drop = [col for col in df.columns if df[col].isnull().mean() > threshold]

print(f"Columnas eliminadas por exceso de nulos: {cols_drop}")
df = df.drop(columns=cols_drop)


Columnas eliminadas por exceso de nulos: []


In [13]:
# Separar columnas numéricas y categóricas
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# Imputar valores faltantes: media para numéricas, moda para categóricas
imputer_num = SimpleImputer(strategy='mean')
imputer_cat = SimpleImputer(strategy='most_frequent')

df[num_cols] = imputer_num.fit_transform(df[num_cols])
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

print("✅ Imputación completada (faltantes reemplazados).")


✅ Imputación completada (faltantes reemplazados).


Codificación de variables categóricas

In [14]:
# Detectar columnas categóricas
cat_cols = df.select_dtypes(include=['object']).columns
print("Columnas categóricas:", list(cat_cols))

# Aplicar one-hot encoding
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
print("✅ Variables categóricas codificadas. Nueva forma:", df_encoded.shape)


Columnas categóricas: ['E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD', 'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']
✅ Variables categóricas codificadas. Nueva forma: (692500, 1032)


Normalización de valores

In [15]:
from sklearn.preprocessing import StandardScaler

# Escalar solo variables numéricas
num_cols = df_encoded.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])

print("✅ Variables numéricas escaladas correctamente.")


✅ Variables numéricas escaladas correctamente.


In [16]:
# Comprobación general
df_encoded.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Columns: 1032 entries, ID to RENDIMIENTO_GLOBAL_medio-bajo
dtypes: bool(1026), float64(6)
memory usage: 709.3 MB


In [17]:
# Vista previa final
df_encoded.head()


,ID,PERIODO_ACADEMICO,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_ACTIVIDAD FISICA Y DEPORTE,E_PRGM_ACADEMICO_ACUICULTURA,E_PRGM_ACADEMICO_ADMINISTRACION,E_PRGM_ACADEMICO_ADMINISTRACION FINANCIERA,...,F_EDUCACIONMADRE_Postgrado,F_EDUCACIONMADRE_Primaria completa,F_EDUCACIONMADRE_Primaria incompleta,F_EDUCACIONMADRE_Secundaria (Bachillerato) completa,F_EDUCACIONMADRE_Secundaria (Bachillerato) incompleta,F_EDUCACIONMADRE_Técnica o tecnológica completa,F_EDUCACIONMADRE_Técnica o tecnológica incompleta,RENDIMIENTO_GLOBAL_bajo,RENDIMIENTO_GLOBAL_medio-alto,RENDIMIENTO_GLOBAL_medio-bajo
0,1.434424,1.294094,0.437002,-0.556223,0.813978,0.060296,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
1,0.527513,1.294094,0.346934,-0.481341,0.508180,0.016142,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
2,-0.652132,0.439801,0.232301,-0.492038,0.729034,0.016142,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False
3,-0.084924,-0.319570,1.771650,-0.941332,-0.171371,-1.072993,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,1.731274,1.294094,0.387874,-0.299484,0.389259,0.457683,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True


In [18]:
# Guardar el dataset limpio para siguientes entregas
df_encoded.to_csv('train_preprocesado.csv', index=False)
print("💾 Archivo guardado: train_preprocesado.csv")


💾 Archivo guardado: train_preprocesado.csv
